In [1]:
!pip install setuptools==69.5.0
import datetime
import torch
import torchvision
import brevitas.nn as qnn
from brevitas.quant.scaled_int import Int8ActPerTensorFloat, Int32Bias


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 893.7/893.7 KB 3.1 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyscaffold 4.4 requires platformdirs<3,>=2, but you have platformdirs 4.2.2 which is incompatible.


In [2]:
WEIGHT_BIT_WIDTH = 8
ACT_BIT_WIDTH = 3

class QuantModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_0 = qnn.QuantConv2d(
            1,
            6,
            kernel_size=3,
            bias=False,
            weight_bit_width=WEIGHT_BIT_WIDTH)
        self.relu_0 = qnn.QuantReLU(bit_width=ACT_BIT_WIDTH)
        self.conv_1 = qnn.QuantConv2d(6, 16, 6,
                                      weight_bit_width=WEIGHT_BIT_WIDTH,
                                      bias=False)
        self.relu_1 = qnn.QuantReLU(bit_width=ACT_BIT_WIDTH)
        self.conv_2 = qnn.QuantConv2d(16, 128, 4,
                                      weight_bit_width=WEIGHT_BIT_WIDTH,
                                      bias=False)
        self.fc1 = qnn.QuantLinear(128, 84,
                                   weight_bit_width=WEIGHT_BIT_WIDTH,
                                   bias=True)
        self.relu_2 = qnn.QuantReLU(bit_width=ACT_BIT_WIDTH)
        self.fc2 = qnn.QuantLinear(84, 10,
                                   weight_bit_width=WEIGHT_BIT_WIDTH,
                                   bias=True)

    def forward(self, x):
        x = self.conv_0(x)
        x = self.relu_0(x)
        x = torch.nn.functional.max_pool2d(x, 2, 2)
        x = self.conv_1(x)
        x = self.relu_1(x)
        x = torch.nn.functional.max_pool2d(x, 2, 2)
        x = self.conv_2(x)
        x = x.view(-1, 128)
        x = self.fc1(x)
        x = self.relu_2(x)
        x = self.fc2(x)
        return x
    

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device)

Using device:  cuda


In [4]:
input_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5,), (0.5,))
])

In [5]:
train_dataset = torchvision.datasets.FashionMNIST('./data', train=True, download=True, transform=input_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = torchvision.datasets.FashionMNIST('./data', train=False, download=True, transform=input_transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

print("Train dataset size: ", len(train_dataset))
print("Val dataset size: ", len(val_dataset))


Train dataset size:  60000
Val dataset size:  10000


In [6]:
model = QuantModel().to(device)
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [7]:
for epoch in range(20):
    print(f"Epoch {epoch}")
    train_loss = 0
    val_loss = 0
    last_loss = 0

    model.train()
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss_value = loss(outputs, labels)
        loss_value.backward()
        optimizer.step()
        train_loss += loss_value.item()
        if i > 0 and i % 100 == 0:
            last_loss = train_loss / 100
            print(f"Batch {i}: Train loss: {last_loss}")
            train_loss = 0

    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss_value = loss(outputs, labels)
            val_loss += loss_value.item()

    val_loss = val_loss / len(val_loader)
    print(f"Val loss: {val_loss}")

torch.save(model.state_dict(), "fashion_mnist_quant.pt")

Epoch 0


/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:1255: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at ../c10/core/TensorImpl.h:1758.)
  return super(Tensor, self).rename(names)


Batch 100: Train loss: 2.324441010951996
Batch 200: Train loss: 2.2975353479385374
Batch 300: Train loss: 2.2933875203132628
Batch 400: Train loss: 2.286476309299469
Batch 500: Train loss: 2.2744906044006346
Batch 600: Train loss: 2.254709367752075
Batch 700: Train loss: 2.2172139477729798
Batch 800: Train loss: 2.146773738861084
Batch 900: Train loss: 2.036548342704773
Val loss: 1.8961950016629165
Epoch 1
Batch 100: Train loss: 1.8155921840667724
Batch 200: Train loss: 1.5335234880447388
Batch 300: Train loss: 1.305894548892975
Batch 400: Train loss: 1.1300686198472976
Batch 500: Train loss: 1.0157148694992066
Batch 600: Train loss: 0.9282804226875305
Batch 700: Train loss: 0.8585860252380371
Batch 800: Train loss: 0.8205051797628403
Batch 900: Train loss: 0.7829470527172089
Val loss: 0.7703432907724077
Epoch 2
Batch 100: Train loss: 0.7255081808567048
Batch 200: Train loss: 0.725473797917366
Batch 300: Train loss: 0.6851366755366325
Batch 400: Train loss: 0.680619804263115
Batch 500:

In [8]:
model.load_state_dict(torch.load("fashion_mnist_quant.pt"))

<All keys matched successfully>

In [9]:
from brevitas.export import export_qonnx
from qonnx.util.cleanup import cleanup as qonnx_cleanup

input_shape = (1, 1, 28, 28)
inp = torch.rand(input_shape)
print(next(val_loader.__iter__())[0].shape)

model.cpu()

qonnx_path = "fashion_mnist_quant.onnx"
export_qonnx(model, inp, export_path=qonnx_path)
qonnx_cleanup(qonnx_path, out_file=qonnx_path)

from finn.util.visualization import showInNetron

showInNetron(qonnx_path)


torch.Size([64, 1, 28, 28])
Serving 'fashion_mnist_quant.onnx' at http://0.0.0.0:8081


In [10]:
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.core.datatype import DataType
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN

finn_path = "fashion_mnist_finn.onnx"

model_for_finn = ModelWrapper(qonnx_path)
model_for_finn = model_for_finn.transform(ConvertQONNXtoFINN())
model_for_finn.save(finn_path)

/home/wojciech/finn-examples/build/finn/deps/qonnx/src/qonnx/transformation/gemm_to_matmul.py:57: UserWarning: The GemmToMatMul transformation only offers explicit support for version 9 of the Gemm node, but the ONNX version of the supplied model is 14. Thus the transformation may fail or return incomplete results.
  warnings.warn(


In [11]:
showInNetron(finn_path)

Stopping http://0.0.0.0:8081
Serving 'fashion_mnist_finn.onnx' at http://0.0.0.0:8081


In [12]:
import finn.core.onnx_exec as oxe

model_for_finn = ModelWrapper(finn_path)
input_name = model_for_finn.graph.input[0].name
input_shape = model_for_finn.get_tensor_shape(input_name)
output_name = model_for_finn.graph.output[0].name
inp = next(val_loader.__iter__())[0][0:1]
inp_dict = {input_name: inp.detach().numpy()}
out_dict = oxe.execute_onnx(model_for_finn, inp_dict)

print(f"FINN output: {out_dict[output_name]}")
print(f"Brevitas output: {model(inp)}")

FINN output: [[-2.574308  -1.9530442 -6.135125  -1.4271387 -3.5321455  3.7578356
  -2.9011254  4.6645055  0.9300549 10.501408 ]]
Brevitas output: tensor([[-2.5743, -1.9530, -6.1351, -1.4271, -3.5321,  3.7578, -2.9011,  4.6645,
          0.9301, 10.5014]], grad_fn=<AddmmBackward0>)
